# Detecting patterns of speciation in the fos- sil record

In this assignment, we use data from the NOW (New and Old Worlds) database of fossil mammals to study patterns of speciation over time and space. In particular, we are interested to know when and where speciation rates have been significantly high. The task is to find which time periods and which places over the history of mammals have given rise to exceptionally high numbers of new species. The phenomenon is known in the evolutionary literature as the “species factory”. Palaeontologists are interested why and in which ways those times and places are special. The role of computational science is to identify and characterize such times and places.
We practice using pandas DataFrames, performing logistic regression and making statistical significance tests in data analysis.

In [14]:
import pandas as pd
import numpy as np
import re

<h3>Exercise 1.</h3>
Data saved in folder as fossils.txt

<h3>Exercise 2.</h3>
Loading from text file into DataFrame, then to csv. File has 68507 rows.

In [17]:
def from_text_to_frame_to_csv(filename):
    #open as usual, declare file object and colum names
    with  open(filename, 'r') as file:
        data = file.readlines()
        columns = data[0].replace('\t',' ').split()
        rows = []
        
        #iterate through rows with regex substitution, split by tabs
        #csv format null is made empty for later use
        for row in data[1:]:
            cells = row.split('\t')
            rows.append(cells)
        
        #combine into dataframe, but mark empty rows as string 'empty'
        #since it will be inserted into csv later
        fossil_frame = pd.DataFrame(rows,columns=columns)
        print('This DataFrame has {} rows'.format(len(fossil_frame)))

In [18]:
filename = 'fossils.txt'

In [324]:
from_text_to_frame_to_csv(filename)

This DataFrame has 68507 rows


<h3>Exercise 3. a)</h3>
a) Remove all rows where LAT = LONG = 0; these occurrences have incorrect coordinates. Drop rows where SPECIES is “sp.” or “indet.”; these occurrences have not been properly identified.

In [335]:
fossil_frame = pd.read_csv('fossils.csv',sep='\t')

In [336]:
def no_latlong_zeros_species_spindet(frame):
    no_zero_latlong = frame[(frame['LAT'] == 0) & (frame['LONG'] == 0)].index
    frame = frame[~(frame.index.isin(no_zero_latlong))]
    frame = frame[~(frame['SPECIES'].isin(['sp.','indet.']))]
    check_sum_latlong = frame[(frame['LAT'] == 0) &(frame['LONG'] == 0)].sum().sum()
    check_sum_species = frame[(frame['SPECIES'].isin(['sp.','indet.']))].sum().sum()
    print(f"Frame filtered of 'LAT' and 'LONG' equals 0. Sum of values containg is: {check_sum_latlong}")
    print(f"Frame filtered of Species 'sp.' and 'indet.' Sum of values containg is: {check_sum_species}")
    return frame.reset_index(drop=True)

In [338]:
fossil_frame = no_latlong_zeros_species_spindet(fossil_frame)

Frame filtered of 'LAT' and 'LONG' equals 0. Sum of values containg is: 0.0
Frame filtered of Species 'sp.' and 'indet.' Sum of values containg is: 0.0


<h3>Exercise 3. b)</h3>
Next we will assign each occurrence to a specific Mammal Neogene
(MN) time unit. Table 1 shows the time boundaries of each time unit.
Assign each occurrence to a correct time unit by calculating the mean of
MIN AGE and MAX AGE. If the mean age of an occurrence is precisely
on the boundary between two time units, assign the occurrence to the
older time unit. If the mean age of an occurrence is outside of the MN
time interval, assign it to a “pre-MN” or “post-MN” category.

In [339]:
def MN_frame(frame):
    labels = ['MQ19','MQ18','MN17','MN16','MN15','MN14','MN13','MN12','MN11','MN10','MN9','MN7-8','MN6','MN5','MN4','MN3','MN2','MN1']
    bins = [0.01,0.85,1.9,2.5,3.55,5,5.3,7.1,7.6,8.9,9.9,11.2,12.85,14.2,16.4,17.2,19.5,21.7,23]
    frame['MEAN_AGE'] = frame[['MAX_AGE','MIN_AGE']].mean(axis=1)
    subset = frame[(frame['MEAN_AGE'] >= .01 ) & (frame['MEAN_AGE'] <= 23 )]['MEAN_AGE']
    frame['MN'] = pd.cut(subset,bins=bins,labels=labels)
    frame['MN'] = frame['MN'].astype(str)
    frame.loc[frame['MEAN_AGE'] < .01,'MN'] = 'post-MN'
    frame.loc[frame['MEAN_AGE'] > 23,'MN'] = 'pre-MN'
    labels.append('pre-MN')
    labels.insert (0, "post-MN") 
    print('Values check out:\n{}'.format(frame.groupby('MN').aggregate({'MEAN_AGE':['max','min']}).reindex(labels)))
    return frame.drop(columns=['MEAN_AGE'])

In [340]:
fossil_frame = MN_frame(fossil_frame)

Values check out:
          MEAN_AGE           
               max        min
MN                           
post-MN   0.005850   0.005850
MQ19      0.850000   0.011500
MQ18      1.865000   0.865000
MN17      2.500000   1.915075
MN16      3.550000   2.520000
MN15      5.000000   3.566500
MN14      5.300000   5.040000
MN13      7.100000   5.350000
MN12      7.600000   7.110000
MN11      8.854500   7.818000
MN10      9.861500   8.938000
MN9      11.167000   9.921900
MN7-8    12.805000  11.215000
MN6      14.200000  13.000000
MN5      16.393625  14.270000
MN4      17.200000  16.405500
MN3      19.478500  17.235000
MN2      21.400000  19.700000
MN1      22.860000  21.735000
pre-MN   65.935000  23.025333
